In [1]:
%reload_ext autoreload
%autoreload 2
from pathlib import Path
from structure_match import StructureMatch

output = StructureMatch.compare_cifs(
    Path("minimal_test/exp.cif"), Path("minimal_test/exp_false_1.cif"),
    resolution_limit=5
)

print(output)

0.006491509402987927


In [2]:
%reload_ext autoreload
%autoreload 2
from pathlib import Path
from structure_match import StructureMatch

output = StructureMatch.generate_hkl_file(Path("minimal_test/exp.cif"))

print(output)

shelx.hkl


In [3]:
%reload_ext autoreload
%autoreload 2
from pathlib import Path
from structure_match import StructureMatch

output = StructureMatch.compare_cif_to_hkl(Path("minimal_test/exp_sx.hkl"), Path("minimal_test/exp.cif"), Path("minimal_test/exp.cif"))

print(output)

0.0035999166479083744
